# 🧊 Peru Minimum Temperature (Tmin) Raster Analysis

This notebook performs comprehensive analysis of Peru's minimum temperature data including:
- Zonal statistics extraction by administrative units
- Climate risk analysis (frost/cold surges)
- Visualization of temperature patterns
- Data preparation for Streamlit app

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import rioxarray as rxr
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up paths
base_path = Path('.').parent
data_path = base_path / 'data'
raster_path = base_path / 'tmin_raster.tif'

print(f"Base path: {base_path}")
print(f"Data path: {data_path}")
print(f"Raster path: {raster_path}")
print(f"Raster exists: {raster_path.exists()}")

## 1. Load and Explore the Raster Data

In [ ]:
# Load raster with rasterio
with rasterio.open(raster_path) as src:
    raster_data = src.read()
    raster_meta = src.meta
    raster_crs = src.crs
    raster_bounds = src.bounds
    raster_transform = src.transform

print(f"Raster shape: {raster_data.shape}")
print(f"Raster CRS: {raster_crs}")
print(f"Raster bounds: {raster_bounds}")
print(f"Number of bands: {raster_data.shape[0]}")
print(f"Data type: {raster_data.dtype}")
print(f"No data value: {raster_meta.get('nodata')}")

# Basic statistics
for band in range(raster_data.shape[0]):
    band_data = raster_data[band]
    valid_data = band_data[band_data != raster_meta.get('nodata', np.nan)]
    print(f"\nBand {band + 1} statistics:")
    print(f"  Min: {np.nanmin(valid_data):.2f}")
    print(f"  Max: {np.nanmax(valid_data):.2f}")
    print(f"  Mean: {np.nanmean(valid_data):.2f}")
    print(f"  Valid pixels: {len(valid_data)}")

In [ ]:
# Visualize the raster
fig, axes = plt.subplots(1, min(3, raster_data.shape[0]), figsize=(15, 5))
if raster_data.shape[0] == 1:
    axes = [axes]

for i, ax in enumerate(axes[:raster_data.shape[0]]):
    band_data = raster_data[i]
    im = ax.imshow(band_data, cmap='coolwarm', vmin=np.nanpercentile(band_data, 1), 
                   vmax=np.nanpercentile(band_data, 99))
    ax.set_title(f'Band {i+1} - Minimum Temperature')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.colorbar(im, ax=ax, label='Temperature (°C)')

plt.tight_layout()
plt.show()

## 2. Download and Load Administrative Boundaries

In [ ]:
# Run the download script
import sys
sys.path.append(str(data_path))

try:
    from download_boundaries import download_peru_boundaries
    download_peru_boundaries()
except Exception as e:
    print(f"Error downloading boundaries: {e}")

In [ ]:
# Load administrative boundaries
boundaries_files = {
    'departments': data_path / 'peru_departments.geojson',
    'districts': data_path / 'peru_districts.geojson'
}

boundaries = {}
for level, file_path in boundaries_files.items():
    if file_path.exists():
        gdf = gpd.read_file(file_path)
        # Ensure CRS is EPSG:4326
        if gdf.crs != 'EPSG:4326':
            gdf = gdf.to_crs('EPSG:4326')
        boundaries[level] = gdf
        print(f"Loaded {level}: {len(gdf)} features")
        print(f"  Columns: {gdf.columns.tolist()}")

# Use the most detailed level available
if 'districts' in boundaries:
    admin_gdf = boundaries['districts'].copy()
    admin_level = 'districts'
else:
    admin_gdf = boundaries['departments'].copy()
    admin_level = 'departments'

print(f"\nUsing {admin_level} for analysis: {len(admin_gdf)} features")

In [ ]:
# Visualize boundaries
fig, ax = plt.subplots(1, 1, figsize=(10, 12))
admin_gdf.plot(ax=ax, facecolor='lightblue', edgecolor='black', alpha=0.7)
ax.set_title(f'Peru Administrative Boundaries ({admin_level.title()})')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

## 3. Zonal Statistics Calculation

In [ ]:
def calculate_zonal_stats(raster_path, geometries, band=1):
    """
    Calculate comprehensive zonal statistics.
    Returns statistics for each geometry.
    """
    stats_list = []
    
    # Define statistics to calculate
    stats_funcs = [
        'count', 'min', 'max', 'mean', 'std',
        'percentile_10', 'percentile_90', 'percentile_25', 'percentile_75'
    ]
    
    print(f"Calculating zonal statistics for band {band}...")
    
    # Calculate zonal stats
    zonal_results = zonal_stats(
        geometries,
        str(raster_path),
        band=band,
        stats=stats_funcs,
        nodata=raster_meta.get('nodata')
    )
    
    # Convert to DataFrame
    stats_df = pd.DataFrame(zonal_results)
    
    # Add custom metrics
    stats_df['range'] = stats_df['max'] - stats_df['min']  # Temperature range
    stats_df['frost_risk'] = (stats_df['min'] < 0).astype(int)  # Binary frost risk
    stats_df['extreme_cold'] = (stats_df['percentile_10'] < -5).astype(int)  # Extreme cold risk
    
    return stats_df

# Calculate zonal statistics for the first band
zonal_stats_df = calculate_zonal_stats(raster_path, admin_gdf.geometry, band=1)

# Combine with administrative data
admin_stats = admin_gdf.copy()
admin_stats = pd.concat([admin_stats.reset_index(drop=True), zonal_stats_df.reset_index(drop=True)], axis=1)

print(f"Zonal statistics calculated for {len(admin_stats)} administrative units")
print(f"Statistics columns: {zonal_stats_df.columns.tolist()}")
print("\nFirst few rows:")
admin_stats[['NAME_CLEAN', 'mean', 'min', 'max', 'std', 'frost_risk']].head()

In [ ]:
# Check for units and rescale if necessary
# If temperatures seem to be scaled (e.g., °C * 10), rescale them
temp_cols = ['min', 'max', 'mean', 'std', 'percentile_10', 'percentile_90', 'percentile_25', 'percentile_75', 'range']

print("Temperature statistics summary:")
print(admin_stats[temp_cols].describe())

# Check if values seem to be scaled (typical range for Peru should be around -10 to 30°C)
mean_temp_range = admin_stats['mean'].max() - admin_stats['mean'].min()
if mean_temp_range > 100:  # Likely scaled by 10 or 100
    print("\nTemperature values appear to be scaled. Rescaling...")
    scale_factor = 10 if mean_temp_range < 1000 else 100
    for col in temp_cols:
        admin_stats[col] = admin_stats[col] / scale_factor
    
    # Recalculate frost risk with corrected values
    admin_stats['frost_risk'] = (admin_stats['min'] < 0).astype(int)
    admin_stats['extreme_cold'] = (admin_stats['percentile_10'] < -5).astype(int)
    
    print(f"Rescaled by factor of {scale_factor}")
    print("Updated temperature statistics:")
    print(admin_stats[temp_cols].describe())

## 4. Analysis and Visualizations

In [ ]:
# 1. Distribution of mean minimum temperatures
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram
admin_stats['mean'].hist(bins=30, ax=ax1, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_xlabel('Mean Minimum Temperature (°C)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Mean Minimum Temperatures')
ax1.axvline(admin_stats['mean'].mean(), color='red', linestyle='--', label=f'Mean: {admin_stats["mean"].mean():.2f}°C')
ax1.legend()

# Box plot
admin_stats.boxplot(column=['mean', 'min', 'max'], ax=ax2)
ax2.set_ylabel('Temperature (°C)')
ax2.set_title('Temperature Statistics Distribution')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 2. Ranking - Coldest and Warmest areas
n_top = min(15, len(admin_stats))

# Sort by mean temperature
coldest = admin_stats.nsmallest(n_top, 'mean')
warmest = admin_stats.nlargest(n_top, 'mean')

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Coldest areas (highest frost risk)
coldest_plot = coldest.plot(x='NAME_CLEAN', y='mean', kind='barh', ax=ax1, color='lightblue')
ax1.set_title(f'Top {n_top} Coldest Areas (Highest Frost Risk)')
ax1.set_xlabel('Mean Minimum Temperature (°C)')
ax1.set_ylabel('')

# Warmest areas (lowest frost risk)
warmest_plot = warmest.plot(x='NAME_CLEAN', y='mean', kind='barh', ax=ax2, color='lightcoral')
ax2.set_title(f'Top {n_top} Warmest Areas (Lowest Frost Risk)')
ax2.set_xlabel('Mean Minimum Temperature (°C)')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

print("COLDEST AREAS (Highest Climate Risk):")
print(coldest[['NAME_CLEAN', 'mean', 'min', 'frost_risk']].to_string())
print("\nWARMEST AREAS (Lowest Climate Risk):")
print(warmest[['NAME_CLEAN', 'mean', 'min', 'frost_risk']].to_string())

In [ ]:
# 3. Static Choropleth Map
fig, ax = plt.subplots(1, 1, figsize=(12, 14))

# Create choropleth map
admin_stats.plot(
    column='mean',
    cmap='RdYlBu_r',
    legend=True,
    ax=ax,
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={'label': 'Mean Minimum Temperature (°C)',
                'orientation': 'horizontal',
                'shrink': 0.8}
)

ax.set_title('Peru: Mean Minimum Temperature by Administrative Unit', fontsize=16, pad=20)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Remove axes ticks for cleaner look
ax.set_xticks([])
ax.set_yticks([])

plt.tight_layout()
plt.savefig(data_path / 'peru_tmin_map.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Climate Risk Analysis

In [ ]:
# Climate risk classification
admin_stats['risk_level'] = 'Low'
admin_stats.loc[admin_stats['mean'] < 5, 'risk_level'] = 'Medium'
admin_stats.loc[admin_stats['mean'] < 0, 'risk_level'] = 'High'
admin_stats.loc[admin_stats['min'] < -10, 'risk_level'] = 'Very High'

# Summary statistics by risk level
risk_summary = admin_stats.groupby('risk_level').agg({
    'NAME_CLEAN': 'count',
    'mean': ['mean', 'min', 'max'],
    'min': ['mean', 'min'],
    'frost_risk': 'sum',
    'extreme_cold': 'sum'
}).round(2)

print("CLIMATE RISK ANALYSIS SUMMARY:")
print(risk_summary)

# Visualize risk distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Risk level distribution
risk_counts = admin_stats['risk_level'].value_counts()
risk_counts.plot(kind='pie', ax=ax1, autopct='%1.1f%%', startangle=90)
ax1.set_title('Distribution of Climate Risk Levels')
ax1.set_ylabel('')

# Temperature vs Risk
risk_colors = {'Low': 'green', 'Medium': 'yellow', 'High': 'orange', 'Very High': 'red'}
for risk in admin_stats['risk_level'].unique():
    data = admin_stats[admin_stats['risk_level'] == risk]
    ax2.scatter(data['mean'], data['min'], label=risk, alpha=0.7, color=risk_colors.get(risk, 'blue'))

ax2.set_xlabel('Mean Minimum Temperature (°C)')
ax2.set_ylabel('Absolute Minimum Temperature (°C)')
ax2.set_title('Temperature Distribution by Risk Level')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Save Results for Streamlit App

In [ ]:
# Save processed data for Streamlit app
output_data = admin_stats.copy()

# Select relevant columns for the app
app_columns = [
    'NAME_CLEAN', 'geometry', 
    'count', 'min', 'max', 'mean', 'std',
    'percentile_10', 'percentile_90', 'percentile_25', 'percentile_75',
    'range', 'frost_risk', 'extreme_cold', 'risk_level'
]

# Add department/region info if available
if 'DEPT_CLEAN' in output_data.columns:
    app_columns.insert(1, 'DEPT_CLEAN')
elif any(col for col in output_data.columns if 'admin' in col.lower() or 'region' in col.lower()):
    region_col = [col for col in output_data.columns if 'admin' in col.lower() or 'region' in col.lower()][0]
    app_columns.insert(1, region_col)

# Filter columns that exist
existing_columns = [col for col in app_columns if col in output_data.columns]
output_data = output_data[existing_columns]

# Save as GeoJSON for Streamlit
output_data.to_file(data_path / 'peru_tmin_analysis.geojson', driver='GeoJSON')

# Save as CSV (without geometry) for easy download
csv_data = output_data.drop(columns=['geometry']).copy()
csv_data.to_csv(data_path / 'peru_tmin_analysis.csv', index=False)

print(f"Saved analysis results:")
print(f"  - GeoJSON: {data_path / 'peru_tmin_analysis.geojson'}")
print(f"  - CSV: {data_path / 'peru_tmin_analysis.csv'}")
print(f"  - Map: {data_path / 'peru_tmin_map.png'}")
print(f"\nData shape: {output_data.shape}")
print(f"Columns: {output_data.columns.tolist()}")

## 7. Summary Statistics and Key Findings

In [ ]:
print("="*60)
print("PERU MINIMUM TEMPERATURE ANALYSIS - KEY FINDINGS")
print("="*60)

print(f"\n📊 DATASET OVERVIEW:")
print(f"  • Administrative units analyzed: {len(admin_stats)}")
print(f"  • Administrative level: {admin_level}")
print(f"  • Raster bands processed: {raster_data.shape[0]}")

print(f"\n🌡️ TEMPERATURE STATISTICS:")
print(f"  • Overall mean minimum temperature: {admin_stats['mean'].mean():.2f}°C")
print(f"  • Coldest area mean: {admin_stats['mean'].min():.2f}°C")
print(f"  • Warmest area mean: {admin_stats['mean'].max():.2f}°C")
print(f"  • Temperature range: {admin_stats['mean'].max() - admin_stats['mean'].min():.2f}°C")

print(f"\n❄️ FROST RISK ANALYSIS:")
print(f"  • Areas with frost risk (min < 0°C): {admin_stats['frost_risk'].sum()} ({admin_stats['frost_risk'].mean()*100:.1f}%)")
print(f"  • Areas with extreme cold risk (p10 < -5°C): {admin_stats['extreme_cold'].sum()} ({admin_stats['extreme_cold'].mean()*100:.1f}%)")

print(f"\n🎯 HIGH-RISK AREAS (Top 5 coldest):")
high_risk = admin_stats.nsmallest(5, 'mean')
for idx, row in high_risk.iterrows():
    print(f"  • {row['NAME_CLEAN']}: {row['mean']:.2f}°C (min: {row['min']:.2f}°C)")

print(f"\n🔥 PRIORITY REGIONS FOR POLICY INTERVENTION:")
priority_areas = admin_stats[admin_stats['risk_level'].isin(['High', 'Very High'])]
print(f"  • {len(priority_areas)} areas require immediate attention")
print(f"  • Focus regions: High-Andean areas and Amazon cold surge zones")

print("\n" + "="*60)